# packages

In [ ]:
!pip install siphon netCDF4 pandas statsmodels geopandas matplotlib cartopy

# Task (MAIN)
Analyze EPD marine water quality data using Python (Pandas, Geopandas, Matplotlib, Scipy/Statsmodels). You are provided two CSV files:

mirs_bay_data.csv: Contains water quality measurements with station identifiers and dates.

station_locations.csv: Contains station identifiers (Station column assumed) and their corresponding Latitude (Lat assumed) and Longitude (Lon assumed).

Perform the following:

Load & Inspect: Load both CSVs into separate DataFrames (df_wq, df_loc). Print column names, data types, and basic info for both. Identify unique stations in df_wq. Identify station/lat/lon columns in df_loc.

Merge Data: Merge df_wq with df_loc based on the common station identifier column (find actual names). Perform a left merge (how='left') to keep all water quality records. Report any water quality stations missing location data after the merge. Use the resulting merged DataFrame (merged_df) for all subsequent analysis.

Clean (merged_df):

Convert date column (find name) to datetime objects and set as index. Handle potential errors (errors='coerce'). Drop rows where date conversion failed.

Identify and convert key parameter columns (Chlorophyll-a (μg/L), Temperature (°C), Salinity (psu), Total Inorganic Nitrogen (mg/L), Orthophosphate Phosphorus (mg/L), E. coli (cfu/100mL), Turbidity (NTU), Dissolved Oxygen (%saturation)) to numeric (pd.to_numeric, errors='coerce').

Report count of NaNs per key parameter column after cleaning.

Spatial (merged_df): Create and display a map of unique station locations using the Lat/Lon columns from the merged data.

Descriptive Stats (merged_df): Calculate and display summary statistics (.describe()) for the key parameters for the entire dataset.

Here is all the data you need:
"mirs_bay_data.csv"
"station_locations.csv"

In [ ]:
# Standard library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import sys

# Add scripts folder to path to find our custom functions
sys.path.append('../scripts')
from analysis_utils import mann_kendall_test, sen_slope

## Data loading

### Subtask:
Load the provided CSV files into pandas DataFrames.

**Reasoning**:
Import pandas and load the two CSV files into pandas DataFrames.

In [ ]:
try:
    df_wq = pd.read_csv('../2_marine_water_quality.csv')
    df_loc = pd.read_csv('../station_locations.csv')
    print("Successfully loaded both CSV files.")
except FileNotFoundError:
    print("Error: One or both CSV files not found.")
    df_wq = None
    df_loc = None
except Exception as e:
    print(f"An error occurred: {e}")
    df_wq = None
    df_loc = None

**Reasoning**:
The two dataframes have been successfully loaded. Now inspect the dataframes by printing column names, data types, and basic info for both.

In [ ]:
if df_wq is not None and df_loc is not None:
    print("Water Quality Data:")
    print("Columns:", df_wq.columns)
    print("Data Types:", df_wq.dtypes)
    print("Info:", df_wq.info())
    print("\nStation Location Data:")
    print("Columns:", df_loc.columns)
    print("Data Types:", df_loc.dtypes)
    print("Info:", df_loc.info())
    unique_stations_wq = df_wq['Station'].unique()
    print(f"\nUnique stations in df_wq:\n{unique_stations_wq}")
    print(f"\nStation/Lat/Lon columns in df_loc:\n{df_loc.columns}")
else:
    print("DataFrames not loaded correctly. Cannot proceed with inspection.")

## Data wrangling

### Subtask:
Merge the water quality data (`df_wq`) with the station location data (`df_loc`).

**Reasoning**:
Merge the two dataframes `df_wq` and `df_loc` based on the 'Station' column using a left merge. Then identify stations in `df_wq` that are missing location data in the merged dataframe.

In [ ]:
# Merge the two dataframes
merged_df = pd.merge(df_wq, df_loc, on='Station', how='left')

# Identify stations missing location data
missing_location = merged_df[merged_df['Lat'].isnull()]['Station'].unique()

# Report the stations missing location data
if missing_location.size > 0:
    print("Stations in df_wq missing location data in df_loc:")
    print(missing_location)
else:
    print("No stations in df_wq are missing location data in df_loc.")

## Data cleaning

### Subtask:
Clean the merged data, including date conversion, handling missing values, and converting relevant columns to numeric types.

**Reasoning**:
Convert the 'Dates' column to datetime, handle errors, and set it as the index. Then, convert specified columns to numeric, handling errors, and count NaN values in those columns.

In [ ]:
# Convert 'Dates' column to datetime and handle errors
merged_df['Dates'] = pd.to_datetime(merged_df['Dates'], errors='coerce')

# Remove rows with NaT values in the 'Dates' column
merged_df = merged_df.dropna(subset=['Dates'])

# Set 'Dates' column as index
merged_df = merged_df.set_index('Dates')

# Identify key parameter columns and convert them to numeric, handling errors
key_parameters = ['Chlorophyll-a (μg/L)', 'Temperature (°C)', 'Salinity (psu)',
                  'Total Inorganic Nitrogen (mg/L)', 'Orthophosphate Phosphorus (mg/L)',
                  'E. coli (cfu/100mL)', 'Turbidity (NTU)', 'Dissolved Oxygen (%saturation)']

for col in key_parameters:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Calculate and report the number of NaN values in each key parameter column
nan_counts = merged_df[key_parameters].isnull().sum()
print("Number of NaN values in each key parameter column:\n", nan_counts)

## Data preparation

### Subtask:
Calculate monthly averages for each key water quality parameter across all stations.

**Reasoning**:
Group the data by month and calculate the monthly averages for the key water quality parameters.

In [ ]:
# Group data by month and calculate monthly averages
key_parameters = ['Chlorophyll-a (μg/L)', 'Temperature (°C)', 'Salinity (psu)',
                  'Total Inorganic Nitrogen (mg/L)', 'Orthophosphate Phosphorus (mg/L)',
                  'E. coli (cfu/100mL)', 'Turbidity (NTU)', 'Dissolved Oxygen (%saturation)']
monthly_averages = merged_df[key_parameters].resample('ME').mean()
display(monthly_averages.head())

## Data visualization

### Subtask:
Visualize the station locations on a map and create time series plots of the monthly averages for each key water quality parameter.

**Reasoning**:
Create a scatter plot of station locations and time series plots for each key water quality parameter.

In [ ]:
# 1. Station Location Map
plt.figure(figsize=(10, 6))
plt.scatter(merged_df['Lon'], merged_df['Lat'], marker='o', alpha=0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Station Locations')
plt.savefig('station_locations_map.png')
plt.show()

# 2. Time Series Plots
key_parameters = ['Chlorophyll-a (μg/L)', 'Temperature (°C)', 'Salinity (psu)',
                  'Total Inorganic Nitrogen (mg/L)', 'Orthophosphate Phosphorus (mg/L)',
                  'E. coli (cfu/100mL)', 'Turbidity (NTU)', 'Dissolved Oxygen (%saturation)']

for param in key_parameters:
    plt.figure(figsize=(10, 6))

    # Compute rolling mean
    rolling_mean = monthly_averages[param].rolling(window=12, min_periods=1).mean()

    # Plot original data
    plt.plot(monthly_averages.index, monthly_averages[param], label='Original Data', alpha=0.6)

    # Plot rolling mean
    plt.plot(monthly_averages.index, rolling_mean, label=f'{12}-Month Rolling Mean', color='red')

    plt.xlabel('Date')
    plt.ylabel(param)
    plt.title(f'Monthly Average of {param} over Time')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'{param.replace("/", "_").replace(" ", "_").replace("(", "_").replace(")", "_")}_timeseries.png')
    plt.show()

## Data analysis

### Subtask:
Perform Mann-Kendall trend analysis and calculate Sen's slope for monthly averages of Temperature, Chlorophyll-a, and Dissolved Oxygen.  Calculate Pearson correlation coefficients between monthly averages of Temperature vs. Chlorophyll-a and Temperature vs. Dissolved Oxygen.

**Reasoning**:
Perform Mann-Kendall trend analysis, calculate Sen's slope, and Pearson correlation coefficients as requested, then compile the results into a summary dictionary.

In [ ]:
results = {}

for param in ['Temperature (°C)', 'Chlorophyll-a (μg/L)', 'Dissolved Oxygen (%saturation)']:
    p_value = mann_kendall_test(monthly_averages[param].dropna().values)
    results[param] = {'Mann-Kendall p-value': p_value,
                      'Significant (p<0.05)': p_value < 0.05,
                      "Sen's Slope": sen_slope(monthly_averages[param].dropna().values)}


# Calculate Pearson correlations
temp_chl_clean_data = monthly_averages[['Temperature (°C)', 'Chlorophyll-a (μg/L)']].dropna()
temp_chl_corr = pearsonr(temp_chl_clean_data['Temperature (°C)'], temp_chl_clean_data['Chlorophyll-a (μg/L)'])

temp_do_clean_data = monthly_averages[['Temperature (°C)', 'Dissolved Oxygen (%saturation)']].dropna()
temp_do_corr = pearsonr(temp_do_clean_data['Temperature (°C)'], temp_do_clean_data['Dissolved Oxygen (%saturation)'])

results['Temperature vs Chlorophyll-a'] = {'Pearson Correlation': temp_chl_corr[0], 'p-value': temp_chl_corr[1]}
results['Temperature vs Dissolved Oxygen'] = {'Pearson Correlation': temp_do_corr[0], 'p-value': temp_do_corr[1]}

for key, value in results.items():
    print(key, value)

## Summary:

### 1. Q&A

* **Are there any water quality stations missing location data?** No, all stations in the water quality dataset have corresponding location information.
* **What is the number of NaN values for each key parameter after cleaning?**  The number of NaN values varies per parameter: Chlorophyll-a (383), Temperature (10), Salinity (31), Total Inorganic Nitrogen (132), Orthophosphate Phosphorus (1427), E. coli (10385), Turbidity (3), Dissolved Oxygen (14).
* **Are there any significant trends in Temperature, Chlorophyll-a, and Dissolved Oxygen based on the Mann-Kendall test?** Temperature and Chlorophyll-a show significant increasing trends (p < 0.05), while Dissolved Oxygen does not show a significant trend.
* **What are the Sen's slopes for Temperature, Chlorophyll-a, and Dissolved Oxygen?**  Temperature: 0.00389, Chlorophyll-a: 0.00197, Dissolved Oxygen: -0.00365
* **What are the Pearson correlation coefficients between Temperature and Chlorophyll-a, and Temperature and Dissolved Oxygen?** Temperature vs Chlorophyll-a: 0.137 (p=0.00387); Temperature vs Dissolved Oxygen: -0.366 (p=2.01e-15)


### 2. Data Analysis Key Findings

* **Missing Data:**  A substantial number of NaN values exist in the 'E. coli (cfu/100mL)' and 'Orthophosphate Phosphorus (mg/L)' parameters (10385 and 1427, respectively).  This suggests potential data collection issues or limitations.
* **Significant Trends:**  Both Temperature and Chlorophyll-a exhibit statistically significant increasing trends over time (Mann-Kendall p-values < 0.05).  Dissolved Oxygen does not show a significant trend.
* **Temperature Correlations:** Temperature shows a weak positive correlation with Chlorophyll-a (Pearson correlation = 0.137, p = 0.00387) and a moderate negative correlation with Dissolved Oxygen (Pearson correlation = -0.366, p = 2.01e-15).  The negative correlation between temperature and dissolved oxygen suggests a potential relationship where higher temperatures could lead to lower dissolved oxygen levels, which is plausible due to decreased solubility of oxygen in warmer water.
* **Sen's Slope:** Sen's slopes provide the magnitude of the trends: Temperature (0.00389), Chlorophyll-a (0.00197), Dissolved Oxygen (-0.00365).


### 3. Insights or Next Steps

* **Investigate Missing Data:**  Prioritize understanding the reasons for the high number of missing values in 'E. coli' and 'Orthophosphate Phosphorus'.  Explore potential imputation methods or investigate the data collection procedures to address the missing data.
* **Further Explore Temperature Relationships:** Given the significant correlations between temperature and other parameters, further investigation is warranted to determine causal relationships and potential ecological implications. Explore more advanced statistical models or time series analysis techniques to investigate those relationships in more detail.